In [1]:
import numpy as np
import librosa
import torch
import laion_clap

c:\Users\Lenovo\conda\envs\clap_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model = laion_clap.CLAP_Module(enable_fusion=False, amodel= 'HTSAT-base')

c:\Users\Lenovo\conda\envs\clap_env\lib\site-packages\torch\functional.py:534: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\TensorShape.cpp:3596.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
model.load_ckpt('../model_checkpoint/music_audioset_epoch_15_esc_90.14 (1).pt')

Load the specified checkpoint ../model_checkpoint/music_audioset_epoch_15_esc_90.14 (1).pt from users.
Load Checkpoint...
logit_scale_a 	 Loaded
logit_scale_t 	 Loaded
audio_branch.spectrogram_extractor.stft.conv_real.weight 	 Loaded
audio_branch.spectrogram_extractor.stft.conv_imag.weight 	 Loaded
audio_branch.logmel_extractor.melW 	 Loaded
audio_branch.bn0.weight 	 Loaded
audio_branch.bn0.bias 	 Loaded
audio_branch.patch_embed.proj.weight 	 Loaded
audio_branch.patch_embed.proj.bias 	 Loaded
audio_branch.patch_embed.norm.weight 	 Loaded
audio_branch.patch_embed.norm.bias 	 Loaded
audio_branch.layers.0.blocks.0.norm1.weight 	 Loaded
audio_branch.layers.0.blocks.0.norm1.bias 	 Loaded
audio_branch.layers.0.blocks.0.attn.relative_position_bias_table 	 Loaded
audio_branch.layers.0.blocks.0.attn.qkv.weight 	 Loaded
audio_branch.layers.0.blocks.0.attn.qkv.bias 	 Loaded
audio_branch.layers.0.blocks.0.attn.proj.weight 	 Loaded
audio_branch.layers.0.blocks.0.attn.proj.bias 	 Loaded
audio_branch

In [4]:
import torch
import torch.nn.functional as F
import numpy as np

def to_torch_tensor(x, device=None, dtype=torch.float32):
    """Convert numpy/torch/sequence to torch.Tensor on `device` with `dtype`.
       Ensures a batch dimension (N, D).
    """
    # detect numpy
    if isinstance(x, np.ndarray):
        t = torch.from_numpy(x)
    elif isinstance(x, torch.Tensor):
        t = x
    else:
        # if it's a list or scalar
        t = torch.tensor(x)
    # ensure float dtype
    if not torch.is_floating_point(t):
        t = t.float()
    # ensure batch dim
    if t.ndim == 1:
        t = t.unsqueeze(0)   # make (1, D)
    elif t.ndim == 0:
        t = t.unsqueeze(0).unsqueeze(0)
    # move to device if given
    if device is not None:
        t = t.to(device=device, dtype=dtype)
    else:
        t = t.to(dtype=dtype)
    return t

# choose device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# get embeddings (your current API calls)
audio_embed = model.get_audio_embedding_from_filelist(
    x=["../Main_DataSet_Creation/data/audio/00/16000.mp3"],
    use_tensor=True
)
text_embed = model.get_text_embedding([
    "Rock "])

# Convert both to torch tensors (handle numpy or torch returns)
audio_t = to_torch_tensor(audio_embed, device=device)
text_t  = to_torch_tensor(text_embed, device=device)

# Optional safety: if shapes differ in dimension size, print helpful info
if audio_t.shape[1] != text_t.shape[1]:
    raise RuntimeError(f"Embedding dimension mismatch: audio {audio_t.shape} vs text {text_t.shape}")

# Normalize (L2) each row so cosine = dot product
audio_n = F.normalize(audio_t, p=2, dim=1)
text_n  = F.normalize(text_t,  p=2, dim=1)

# If both are batches you want pairwise similarity for each row:
# - if both batches same length: per-row similarity
# - if audio batch and single text row: compute each audio vs that text
if audio_n.shape[0] == text_n.shape[0]:
    sim = F.cosine_similarity(audio_n, text_n, dim=1)  # shape (N,)
else:
    # broadcast: compare each audio row to first text row
    sim = F.cosine_similarity(audio_n, text_n.expand(audio_n.shape[0], -1), dim=1)

print("Similarity vector:", sim)
print("First similarity (scalar):", float(sim[0].item()))

Similarity vector: tensor([0.3252], device='cuda:0', grad_fn=<SumBackward1>)
First similarity (scalar): 0.3251603841781616


In [5]:
import pandas as pd
from typing import List, Tuple
def to_torch_tensor(x, device=None, dtype=torch.float32):
    """Convert numpy or torch to torch tensor on device with float dtype."""
    if isinstance(x, np.ndarray):
        t = torch.from_numpy(x)
    elif isinstance(x, torch.Tensor):
        t = x
    else:
        raise TypeError(f"Unsupported type {type(x)}")
    if t.dtype in (torch.float64, torch.float16):
        t = t.to(dtype)
    if device is not None:
        t = t.to(device)
    return t

In [6]:
def l2_normalize(t: torch.Tensor, eps: float = 1e-12) -> torch.Tensor:
    """L2-normalize along last dim."""
    denom = t.norm(p=2, dim=-1, keepdim=True).clamp_min(eps)
    return t / denom

def ensure_2d(t: torch.Tensor) -> torch.Tensor:
    """Make sure tensor is 2D: (N, D)"""
    if t.ndim == 1:
        return t.unsqueeze(0)
    return t

In [7]:
def embed_audio_single(model, audio_path: str, device: torch.device):
    """Return audio embedding as torch tensor (1, D). Handles numpy or torch returns."""
    emb = model.get_audio_embedding_from_filelist(x=[audio_path], use_tensor=True)
    # CLAP sometimes returns numpy array or torch; handle both
    if isinstance(emb, np.ndarray):
        emb_t = to_torch_tensor(emb, device=device)
    else:
        emb_t = emb.to(device)
    emb_t = ensure_2d(emb_t.float())  # (1, D)
    return emb_t

In [8]:
def embed_texts(model, texts: List[str], device: torch.device, chunk_size:int = 256):
    """
    Embed a list of texts. CLAP's get_text_embedding often accepts list directly.
    This handles returns that might be numpy arrays and chunks if the list is large.
    Returns (N, D) torch tensor on device.
    """
    # try directly first
    try:
        te = model.get_text_embedding(texts)
        if isinstance(te, np.ndarray):
            te_t = to_torch_tensor(te, device=device)
        else:
            te_t = te.to(device)
        return ensure_2d(te_t.float())
    except Exception:
        # chunked fallback (safer if the model method fails for huge lists)
        out = []
        for i in range(0, len(texts), chunk_size):
            chunk = texts[i:i+chunk_size]
            te = model.get_text_embedding(chunk)
            if isinstance(te, np.ndarray):
                te_t = to_torch_tensor(te, device=device)
            else:
                te_t = te.to(device)
            out.append(ensure_2d(te_t.float()))
        return torch.cat(out, dim=0)

In [9]:
def rank_texts_for_audio(
    model,
    audio_path: str,
    candidate_texts: List[str],
    top_k: int = 5,
    batch_size: int = 256,   # controls memory usage
    device=None
) -> List[Tuple[str, float]]:
    """
    Rank candidate_texts by cosine similarity to audio_path. Returns top_k tuples (text, score).
    Score is cosine similarity in [-1,1].
    """
    if device is None:
        device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

    # ---- Audio embedding ----
    audio_emb = embed_audio_single(model, audio_path, device=device)    # (1, D)
    audio_n = l2_normalize(audio_emb)  # normalize

    all_scores = []
    all_texts = []

    # ---- Process text embeddings in batches ----
    for start in range(0, len(candidate_texts), batch_size):
        end = start + batch_size
        batch_texts = candidate_texts[start:end]

        text_emb = embed_texts(model, batch_texts, device=device)   # (B, D)
        text_n = l2_normalize(text_emb)

        # similarity (1, B) → (B,)
        sims = (audio_n @ text_n.T).squeeze(0).detach().cpu().numpy()

        all_scores.extend(sims.tolist())
        all_texts.extend(batch_texts)

        # free GPU memory after each batch
        del text_emb, text_n, sims
        torch.cuda.empty_cache()

    # ---- Rank across all texts ----
    all_scores = np.array(all_scores)
    top_idx = np.argsort(-all_scores)[:top_k]

    return [(all_texts[i], float(all_scores[i])) for i in top_idx]


In [10]:
audio_path = "../Main_DataSet_Creation/data/audio/00/16000.mp3"   # your path
captions_df = pd.read_csv("../Main_DataSet_Creation/data/captions_checkpoint.csv")  # or your caption file
candidate_texts = captions_df["caption"].astype(str).tolist()

In [45]:
import gc, torch
gc.collect()
torch.cuda.empty_cache()

In [11]:
print(torch.cuda.memory_summary(device=None, abbreviated=False))

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |   1015 MiB |   1060 MiB |   1501 MiB | 497584 KiB |
|       from large pool |    896 MiB |    914 MiB |   1069 MiB | 177550 KiB |
|       from small pool |    119 MiB |    164 MiB |    431 MiB | 320033 KiB |
|---------------------------------------------------------------------------|
| Active memory         |   1015 MiB |   1060 MiB |   1501 MiB | 497584 KiB |
|       from large pool |    896 MiB |    914 MiB |   1069 MiB |

In [12]:
top = rank_texts_for_audio(model, audio_path, candidate_texts, top_k=10, device=device, batch_size=256)
print("Top matches among captions:")
for t, s in top:
    print(f"{s:.4f}  {t[:220].replace('/n',' ')}")

Top matches among captions:
0.5054  A vigorous and aggressive instrumental Hard Rock piece with a loud electric guitar and drums playing alongside a deep male vocal roar. Keywords: hard rock, aggressive, instrumental, vigorous.
0.5015  A fast and energetic rock piece with drums, guitar, electric guitar, and synthesizer sounds. Electric guitar solo, distorted, heavy, loud, and melodic. Key is E major, time signature is 4/4, tempo is 157.89
0.4952  A fast-paced, aggressive instrumental rock piece with drums and electric guitar playing a loud, distorted beat. Keywords: rock, aggressive, instrumental, fast-paced.
0.4947  A fast-paced, aggressive instrumental rock piece with drums and electric guitar playing a hard-hitting beat and a distorted bass line. Keywords: rock, aggressive, instrumental, fast-paced.
0.4915  A passionate electronic rock piece with heavy drums and distorted guitars playing an energetic melody. Keywords: electronic, rock, drums, guitar.
0.4909  A loud, aggressive instr